In [8]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [24]:
from accounts.models import User,Facility,House

In [78]:
from res_demo.models import Restaurant,Pizza,Topping
from django.db.models import Q
from pprint import pprint

In [18]:
# ALL

In [21]:
print(House.objects.all())
print(House.objects.all().query)

SELECT "accounts_house"."id", "accounts_house"."owner_id", "accounts_house"."address" FROM "accounts_house"


In [22]:
print(Facility.objects.all())
print(Facility.objects.all().query)

<QuerySet [<Facility: Table>, <Facility: Bed>, <Facility: Lamp>, <Facility: jacuzzi>, <Facility: Lift>, <Facility: Playroom>]>
SELECT "accounts_facility"."id", "accounts_facility"."name" FROM "accounts_facility"


In [23]:
print(Pizza.objects.all())
print(Pizza.objects.all().query)

<QuerySet [<Pizza: margarita (Cheese)>, <Pizza: Overloaded (Cheese, Jalapeno, Paneer, Olives, Mushroom)>, <Pizza: PineapplePizza (Pineapple, Cheese)>, <Pizza: HamPizza (Ham, Cheese, Jalapeno, Olives, Mushroom)>, <Pizza: SeaFood (Cheese, Jalapeno, Prawns, Olives, Mushroom)>]>
SELECT "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_pizza"


In [35]:
# Filter query

In [37]:
User.objects.all()

<QuerySet [<User: test>, <User: u2>, <User: admin>]>

In [42]:
print(User.objects.filter(username="test").get())
print(User.objects.filter(username="test").query)

test
SELECT "accounts_user"."id", "accounts_user"."password", "accounts_user"."last_login", "accounts_user"."is_superuser", "accounts_user"."username", "accounts_user"."first_name", "accounts_user"."last_name", "accounts_user"."email", "accounts_user"."is_staff", "accounts_user"."is_active", "accounts_user"."date_joined", "accounts_user"."region" FROM "accounts_user" WHERE "accounts_user"."username" = test


In [65]:
print(Pizza.objects.filter(toppings__name='Cheese').values_list("name"),end="\n\n")
print(Pizza.objects.filter(toppings__name='Cheese').query)

<QuerySet [('margarita',), ('Overloaded',), ('PineapplePizza',), ('HamPizza',), ('SeaFood',)]>

SELECT "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_pizza" INNER JOIN "res_demo_pizza_toppings" ON ("res_demo_pizza"."id" = "res_demo_pizza_toppings"."pizza_id") INNER JOIN "res_demo_topping" ON ("res_demo_pizza_toppings"."topping_id" = "res_demo_topping"."id") WHERE "res_demo_topping"."name" = Cheese


In [95]:
print(Pizza.objects.exclude(toppings__name='Ham').values_list("name"),end="\n\n")
print(Pizza.objects.filter(toppings__name='Ham').values_list("name"),end="\n\n")

print(Pizza.objects.exclude(toppings__name='Ham').query,end="\n\n")
print(Pizza.objects.filter(toppings__name='Ham').query)

<QuerySet [('margarita',), ('Overloaded',), ('PineapplePizza',), ('SeaFood',)]>

<QuerySet [('HamPizza',)]>

SELECT "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_pizza" WHERE NOT ("res_demo_pizza"."id" IN (SELECT U1."pizza_id" FROM "res_demo_pizza_toppings" U1 INNER JOIN "res_demo_topping" U2 ON (U1."topping_id" = U2."id") WHERE U2."name" = Ham))

SELECT "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_pizza" INNER JOIN "res_demo_pizza_toppings" ON ("res_demo_pizza"."id" = "res_demo_pizza_toppings"."pizza_id") INNER JOIN "res_demo_topping" ON ("res_demo_pizza_toppings"."topping_id" = "res_demo_topping"."id") WHERE "res_demo_topping"."name" = Ham


In [102]:
print(House.objects.filter(address__icontains="q"),end="\n\n")
print(House.objects.filter(address__icontains="q").query)

<QuerySet [<House: u2_aadfq>, <House: u2_aadfbq>]>

SELECT "accounts_house"."id", "accounts_house"."owner_id", "accounts_house"."address" FROM "accounts_house" WHERE "accounts_house"."address" LIKE %q% ESCAPE '\'


In [91]:
# Q
print(Restaurant.objects.filter(best_pizza__name="margarita"),end="\n\n")
print(Restaurant.objects.filter(best_pizza__name="margarita").query,"\n\n")

print(Restaurant.objects.filter(Q(best_pizza__name="margarita") | Q(best_pizza__name="Overloaded")),end="\n\n")
print(Restaurant.objects.filter(Q(best_pizza__name="margarita")| Q(best_pizza__name="Overloaded")).query)

<QuerySet [<Restaurant: Joes>]>

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id" FROM "res_demo_restaurant" INNER JOIN "res_demo_pizza" ON ("res_demo_restaurant"."best_pizza_id" = "res_demo_pizza"."id") WHERE "res_demo_pizza"."name" = margarita 


<QuerySet [<Restaurant: Joes>, <Restaurant: VegResturant>]>

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id" FROM "res_demo_restaurant" INNER JOIN "res_demo_pizza" ON ("res_demo_restaurant"."best_pizza_id" = "res_demo_pizza"."id") WHERE ("res_demo_pizza"."name" = margarita OR "res_demo_pizza"."name" = Overloaded)


In [ ]:
#Reverse Query

In [57]:
u = User.objects.get(username = "test")
pizza = Pizza.objects.get(name="margarita")

In [58]:
u,pizza

(<User: test>, <Pizza: margarita (Cheese)>)

In [62]:
print(pizza.restaurants.all(),end="\n\n")

print(Restaurant.objects.filter(pizzas=pizza))

<QuerySet [<Restaurant: Joes>, <Restaurant: VegResturant>, <Restaurant: Pomos>]>

<QuerySet [<Restaurant: Joes>, <Restaurant: VegResturant>, <Restaurant: Pomos>]>


In [63]:
print(Restaurant.objects.filter(pizzas=pizza).query,end="\n\n")
print(pizza.restaurants.all().query)

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id" FROM "res_demo_restaurant" INNER JOIN "res_demo_restaurant_pizzas" ON ("res_demo_restaurant"."id" = "res_demo_restaurant_pizzas"."restaurant_id") WHERE "res_demo_restaurant_pizzas"."pizza_id" = 1

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id" FROM "res_demo_restaurant" INNER JOIN "res_demo_restaurant_pizzas" ON ("res_demo_restaurant"."id" = "res_demo_restaurant_pizzas"."restaurant_id") WHERE "res_demo_restaurant_pizzas"."pizza_id" = 1


In [43]:
#SELECT RELATED

In [50]:
print(Restaurant.objects.select_related("best_pizza"),end="\n\n")
print(Restaurant.objects.select_related("best_pizza").query,end="\n\n")
print(Restaurant.objects.all().query)

<QuerySet [<Restaurant: Joes>, <Restaurant: VegResturant>, <Restaurant: Pomos>]>

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id", "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_restaurant" INNER JOIN "res_demo_pizza" ON ("res_demo_restaurant"."best_pizza_id" = "res_demo_pizza"."id")

SELECT "res_demo_restaurant"."id", "res_demo_restaurant"."name", "res_demo_restaurant"."best_pizza_id" FROM "res_demo_restaurant"


In [10]:
# Prefetch related

In [92]:
print(Pizza.objects.prefetch_related("toppings"),end="\n\n")
print(Pizza.objects.prefetch_related("toppings").query)

<QuerySet [<Pizza: margarita (Cheese)>, <Pizza: Overloaded (Cheese, Jalapeno, Paneer, Olives, Mushroom)>, <Pizza: PineapplePizza (Pineapple, Cheese)>, <Pizza: HamPizza (Ham, Cheese, Jalapeno, Olives, Mushroom)>, <Pizza: SeaFood (Cheese, Jalapeno, Prawns, Olives, Mushroom)>]>

SELECT "res_demo_pizza"."id", "res_demo_pizza"."name" FROM "res_demo_pizza"


In [14]:
Pizza.objects.all()

<QuerySet [<Pizza: margarita (Cheese)>, <Pizza: Overloaded (Cheese, Jalapeno, Paneer, Olives, Mushroom)>, <Pizza: PineapplePizza (Pineapple, Cheese)>, <Pizza: HamPizza (Ham, Cheese, Jalapeno, Olives, Mushroom)>, <Pizza: SeaFood (Cheese, Jalapeno, Prawns, Olives, Mushroom)>]>